## **데이터 불러오기**

In [11]:
import pandas as pd

train_set = pd.read_csv("sentiment-analysis/data/감성대화망뭉치(병합).csv", index_col=0) #첫 번째 열이 인덱스 열일 때 Unnamed: 0 빼기
train_set.sample(n=5)

FileNotFoundError: [Errno 2] No such file or directory: 'sentiment-analysis/data/감성대화망뭉치(병합).csv'

In [8]:
# train_set = train_set[['감정_대분류','사람문장1']]
# train_set.columns = ['emotion', 'sentence']  # 칼럼이름 변경

# train_set.head()

**감정을 정수 라벨로 변경**

In [ ]:
# 감정을 정수 라벨로 변경
emotions = {'기쁨': 0, '불안': 1, '당황': 2, '슬픔': 3, '분노': 4, '상처': 5}

train_set['emotion'] = train_set.emotion.map(emotions)

train_set.sample(n=5)
# train_set.loc[(train_set['emotion'] == "기쁨"), 'emotion'] = 0  #기쁨 => 0
# train_set.loc[(train_set['emotion'] == "불안"), 'emotion'] = 1  #불안 => 1
# train_set.loc[(train_set['emotion'] == "당황"), 'emotion'] = 2  #당황 => 2
# train_set.loc[(train_set['emotion'] == "슬픔"), 'emotion'] = 3  #슬픔 => 3
# train_set.loc[(train_set['emotion'] == "분노"), 'emotion'] = 4  #분노 => 4
# train_set.loc[(train_set['emotion'] == "상처"), 'emotion'] = 5  #상처 => 5

__Korean BERT pre-trained cased (KoBERT) for Huggingface Transformers__
<br>
koBERT Hugging Face 토크나이저 및  Requirements 설치

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install sentencepiece
!pip install transformers
!pip install torch

!pip install mxnet
!pip install gluonnlp pandas tqdm

In [ ]:
# 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
 
#kobert
from kobert_tokenizer import KoBERTTokenizer

# transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 사용시 필요
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = torch.device("cuda:0")

**토크나이저, pretrained 모델, vocabulary 로드**

In [12]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters(KoBERT finetuning 베에스 라인을 그대로 따랐음)
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

NameError: name 'KoBERTTokenizer' is not defined

### **KOBert 클래스 정의**

In [ ]:
# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

In [ ]:
# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, #클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

**정확도 계산**

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 입력에 대한 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

**데이터 셋 피팅**

In [ ]:
# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류 
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=4)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=2)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=2)

학습 코드

# 학습
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

**모델 정보**

In [ ]:
# 모델 정보
print(type(model), model, sep="\n\n")

In [ ]:
# 상대경로 안됨?
# torch.save(model, './model/sa-tutorial1.pt')
# torch.save(model.state_dict(), './model/sa-tutorial1_StateDict.pt')

torch.save(model, f'/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1_StateDict.pt')

In [ ]:
# 학습된 모델 저장
import os

model_path1 = '/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1.pt'
model_path2 = '/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1_StateDict.pt'

size1 = os.path.getsize(model_path1) / (1024*1024) # mb 단위
size2 = os.path.getsize(model_path2) / (1024*1024) 
print(f"Model size: {size1:.2f} MB")
print(f"Model_StateDict size: {size2:.2f} MB")